<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [7]:
import numpy as np
import pandas as pd
import psycopg2

In [8]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 
# Данный для подключения были заменены необходимо внести корректные данные
DBNAME = 'база данных'
USER = 'пользователь '
PASSWORD = 'Пароль'
HOST = 'IP address'
PORT = 5432 

In [9]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [10]:
# текст запроса
query_3_1 = f'''
SELECT count(*) FROM vacancies
'''


In [11]:
# результат запроса
df_vacancies = pd.read_sql_query(query_3_1, connection)
print(df_vacancies)

C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_23508\1909379053.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_vacancies = pd.read_sql_query(query_3_1, connection)


   count
0  49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [12]:
# текст запроса
query_3_2 = f'''
SELECT count(*) FROM employers
'''


In [13]:
# результат запроса
df_employers = pd.read_sql_query(query_3_2, connection)
print(df_employers)

   count
0  23501


C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_23508\946347788.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_employers = pd.read_sql_query(query_3_2, connection)


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [14]:
# текст запроса
query_3_3 = f'''
SELECT count(*) FROM areas
'''

In [15]:
# результат запроса
df_areas = pd.read_sql_query(query_3_3, connection)
print(df_areas)

   count
0   1362


C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_23508\2878732808.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_areas = pd.read_sql_query(query_3_3, connection)


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [16]:
# текст запроса
query_3_4 = f'''
SELECT count(*) FROM industries
'''


In [17]:
# результат запроса
df_industries = pd.read_sql_query(query_3_3, connection)
print(df_industries)

   count
0   1362


C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_23508\175086551.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_industries = pd.read_sql_query(query_3_3, connection)


<h3> выводы по предварительному анализу данных </h3>

<p>Если брать выводы только по количеству строк, то видно, что таблицы разнообразные по длинне.<br>
Вероятно таблица с вакансиями основаня таблица содержить больше всего записей.
Остальные таблицы связные таблицы.</p>

<b><i>Дополнительно.</i></b><br>
Для проверки сделанного вывода ниже был выполниен дополнительный запрос в базу.
<p>Который отобразит тип данных название столбцов у перечисленных выше таблиц.<br>
Видно что в таблице вакансий есть текстовые поля и поля кулючи которые по названию совпадают с таблицами (<i> employers, areas </i>)
что подтверждает что эти таблицы связные.<p>
  



***

In [18]:
# дополнительный предварительный анализ к выводу
query_3_final = f'''SELECT table_schema, table_name, column_name, data_type FROM information_schema.columns
WHERE table_schema = 'public' AND (table_name = 'industries' OR table_name = 'areas' OR table_name = 'employers' OR table_name = 'vacancies')
'''
# результат запроса
df_industries = pd.read_sql_query(query_3_final, connection)
print(df_industries)


C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_23508\3841190620.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_industries = pd.read_sql_query(query_3_final, connection)


   table_schema  table_name  column_name          data_type
0        public   employers           id            integer
1        public   employers         name               text
2        public   employers         area            integer
3        public  industries           id  character varying
4        public  industries         name               text
5        public   vacancies           id            integer
6        public   vacancies         name               text
7        public   vacancies   key_skills               text
8        public   vacancies     schedule               text
9        public   vacancies   experience               text
10       public   vacancies   employment               text
11       public   vacancies  salary_from            integer
12       public   vacancies    salary_to            integer
13       public   vacancies      area_id            integer
14       public   vacancies  employer_id            integer
15       public       areas           id

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [19]:
# текст запроса
query_4_1 = f'''SELECT
                a.name,
                count(v.id) as cnt

            FROM
                vacancies v
                join areas a ON a.id = v.area_id 
            GROUP BY a.name
            order by 2 DESC
        '''

In [20]:
# результат запроса

df_4_1 = pd.read_sql_query(query_4_1, connection)
print(df_4_1)

C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_23508\4134734627.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_4_1 = pd.read_sql_query(query_4_1, connection)


                name   cnt
0             Москва  5333
1    Санкт-Петербург  2851
2              Минск  2112
3        Новосибирск  2006
4             Алматы  1892
..               ...   ...
764       Тарко-Сале     1
765    Новоаннинский     1
766            Бирск     1
767           Сасово     1
768             Анна     1

[769 rows x 2 columns]


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [21]:
# текст запроса
query_4_2 = f'''SELECT
                count(v.id) as cnt
            FROM
                vacancies v 
            WHERE (salary_from is not NULL or salary_to is not NULL)
        '''

In [22]:
# результат запроса
df_4_2 = pd.read_sql_query(query_4_2, connection)
print(df_4_2)


     cnt
0  24073


C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_23508\1648207881.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_4_2 = pd.read_sql_query(query_4_2, connection)


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [23]:
# текст запроса
query_4_3 = f'''SELECT
                ROUND(AVG(salary_to)) as max,
                ROUND(AVG(salary_from)) as min
            FROM
                vacancies v 
         '''

In [24]:
# результат запроса
df_4_3 = pd.read_sql_query(query_4_3, connection)
print(df_4_3)


        max      min
0  110537.0  71065.0


C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_23508\78768766.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_4_3 = pd.read_sql_query(query_4_3, connection)


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [25]:
# текст запроса
query_4_4 = f'''with schedule_tmp  as (SELECT
                DISTINCT v.schedule  
            FROM
                vacancies v),

            employment_tmp as (SELECT
                DISTINCT v.employment  
            FROM
                vacancies v),
            s_e_tmp as (
            select * FROM employment_tmp join schedule_tmp ON TRUE)

            select 
                set.employment,
                set.schedule,
                count(v.id)
            FROM
                vacancies v JOIN s_e_tmp set ON (set.employment = v.employment AND set.schedule = v.schedule)
            GROUP BY set.employment, set.schedule
            ORDER BY 3 DESC

         '''

In [26]:
# результат запроса
df_4_4 = pd.read_sql_query(query_4_4, connection)
print(df_4_4)

             employment          schedule  count
0      Полная занятость       Полный день  35367
1      Полная занятость  Удаленная работа   7802
2      Полная занятость     Гибкий график   1593
3   Частичная занятость  Удаленная работа   1312
4      Полная занятость    Сменный график    940
5            Стажировка       Полный день    569
6      Полная занятость    Вахтовый метод    367
7   Частичная занятость       Полный день    347
8   Частичная занятость     Гибкий график    312
9      Проектная работа       Полный день    141
10     Проектная работа  Удаленная работа    133
11           Стажировка     Гибкий график    116
12  Частичная занятость    Сменный график    101
13           Стажировка  Удаленная работа     64
14     Проектная работа     Гибкий график     18
15           Стажировка    Сменный график     12
16     Проектная работа    Вахтовый метод      2
17     Проектная работа    Сменный график      1


C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_23508\2038540691.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_4_4 = pd.read_sql_query(query_4_4, connection)


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [27]:
# текст запроса
query_4_5 = f'''with exp_tmp as (select 
                v.experience    
            FROM
                vacancies v
            GROUP BY experience)
            
            SELECT
                e.experience,
                count(v.experience)
            FROM
                vacancies v
                JOIN exp_tmp e ON e.experience = v.experience
                GROUP BY e.experience
                ORDER BY 2
         '''


In [28]:
# результат запроса
df_4_5 = pd.read_sql_query(query_4_5, connection)
print(df_4_5)


           experience  count
0         Более 6 лет   1337
1           Нет опыта   7197
2       От 3 до 6 лет  14511
3  От 1 года до 3 лет  26152


C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_23508\1045726932.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_4_5 = pd.read_sql_query(query_4_5, connection)


***

<h3> выводы по детальному анализу данных </h3>

Проведя предложенный более детальный анализ таблиц мы видим:
<p>
1. Количество вакансий больше в крупных городах и уменьшается к регионалным.<br>
2. Около половины соискателей заполняет поле с желаемыми границами зарплаты <br>
3. Около 70% соискателей предпочитают работать полный рабочий день с полной занятостью <br>
Около 15% соискателей предпочитают работать полный рабочий день удаленно. И 15% другие виды занятости <br>
4. Более ~70% соискателей  люди которые получили небольшой опыт или вообще его не имели <br>
~28% людей со средним опытом работы. И лишь 2% людей с большим опытом. <br>
Что позволяет нам сделать вывод, что опытные работники не стремятся менять работу</p>


# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [29]:
# текст запроса
query_5_1 = f'''SELECT
                e.name,
                count(v.employer_id) as cnt
            FROM
                employers e
                JOIN vacancies v ON v.employer_id = e.id
            GROUP BY 1
            ORDER BY 2 DESC 
            LIMIT 5
         '''

In [30]:
# результат запроса
df_5_1 = pd.read_sql_query(query_5_1, connection)
print(df_5_1.iloc[0],"\n", df_5_1.iloc[4])

name    Яндекс
cnt       1933
Name: 0, dtype: object 
 name    Газпром нефть
cnt               331
Name: 4, dtype: object


C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_23508\3009691854.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5_1 = pd.read_sql_query(query_5_1, connection)


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [31]:
# текст запроса
query_5_2 = f'''with e_area_count as (SELECT
                e.area,
                count(e.id) as employer_cnt
            FROM
                employers e
            GROUP BY 1),

            v_area_count as (SELECT
                v.area_id,
                count(v.id) as vac_cnt
            FROM
                vacancies v
            GROUP BY 1)


        SELECT
            employer_cnt, vac_cnt, name
        FROM
            areas a 
            LEFT JOIN e_area_count e ON e.area = a.id
            LEFT JOIN v_area_count v ON v.area_id = a.id
        WHERE employer_cnt IS NOT NULL 
            ORDER BY 2 DESC, 1
         '''


In [32]:
# результат запроса
df_5_2 = pd.read_sql_query(query_5_2, connection)
print(df_5_2)
print(df_5_2[df_5_2["vac_cnt"].isna()][['name', 'employer_cnt']].tail(1)['name'])

C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_23508\1510520429.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5_2 = pd.read_sql_query(query_5_2, connection)


     employer_cnt  vac_cnt                                   name
0               1      NaN                       Хабаровский край
1               1      NaN  Радужный (Ханты-Мансийский АО - Югра)
2               1      NaN                                 Рыбное
3               1      NaN                                  Венев
4               1      NaN                         Усть-Ордынский
..            ...      ...                                    ...
649             2      1.0        Радужный (Владимирская область)
650             2      1.0                               Белорецк
651             2      1.0                                 Балхаш
652             3      1.0                                    Бор
653             3      1.0                                  Китай

[654 rows x 3 columns]
185    Россия
Name: name, dtype: object


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [33]:
# текст запроса
query_5_3 = f'''SELECT
                e.name,
                count(DISTINCT v.area_id) as area_cnt 
            FROM
                vacancies v 
                JOIN employers e ON v.employer_id = e.id
            GROUP BY e.name
            ORDER BY area_cnt DESC
            
            '''



In [34]:
# результат запроса
df_5_3 = pd.read_sql_query(query_5_3, connection)
print(df_5_3)

C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_23508\1496354524.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5_3 = pd.read_sql_query(query_5_3, connection)


                         name  area_cnt
0                      Яндекс       181
1                  Ростелеком       152
2                  Спецремонт       116
3      Поляков Денис Иванович        88
4                   ООО ЕФИН         71
...                       ...       ...
14761                  UniSol         1
14762            UNISTORY LLC         1
14763                   UNIT6         1
14764     United Distribution         1
14765              UNITEL LLC         1

[14766 rows x 2 columns]


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [35]:
# текст запроса
query_5_4 = f'''with conuter as (SELECT
                count( ei.industry_id) 

            FROM
                EMPLOYERS_INDUSTRIES ei
                right join employers e ON e.id = ei.employer_id 
            where ei.employer_id IS NULL
            GROUP BY e.id)
            
            select 
                count(*)
            FROM 
                conuter

         '''


In [36]:
# результат запроса
df_5_4 = pd.read_sql_query(query_5_4, connection)
print(df_5_4)

   count
0   8419


C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_23508\3675082845.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5_4 = pd.read_sql_query(query_5_4, connection)


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [37]:
# текст запроса
query_5_5 = f'''with conuter as (SELECT
                e.name,
                count( ei.industry_id) 

            FROM
                EMPLOYERS_INDUSTRIES ei
                right join employers e ON e.id = ei.employer_id 
            GROUP BY e.id)
            
            select 
                name
            FROM 
                conuter
            WHERE count = 4
            ORDER BY name
            OFFSET 2 
            LIMIT 1
         '''

In [38]:
# результат запроса
df_5_5 = pd.read_sql_query(query_5_5, connection)
print(df_5_5)

   name
0  2ГИС


C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_23508\1922466895.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5_5 = pd.read_sql_query(query_5_5, connection)


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [39]:
# текст запроса
query_5_6 = f'''SELECT
                count( ei.employer_id) 
            FROM
                EMPLOYERS_INDUSTRIES ei
                JOIN INDUSTRIES i ON ei.industry_id = i.id
            WHERE i.name = 'Разработка программного обеспечения'
         '''



In [40]:
# результат запроса
df_5_6 = pd.read_sql_query(query_5_6, connection)
print(df_5_6)

   count
0   3553


C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_23508\4288990653.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5_6 = pd.read_sql_query(query_5_6, connection)


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [41]:
# код для получения списка городов-милионников

# Данный способ позволяет получить списко городов миллионников с википедии до тех пор пока страница википедии не поменяется.
from bs4 import BeautifulSoup
import requests
def rebuild_table_as_list(html_table):
    '''
    Функция преобразует строки таблицы в переменную типа list
    и записывает каждую строку в общую переменную типа list 
    '''
    data = []
    for element in html_table:
        sub_data = []
        for sub_element in element:
            try:
                if sub_element.get_text() != "\n":
                    sub_data.append(sub_element.get_text())
            except:
                continue
        data.append(sub_data)
    return data

url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(url)
soup = BeautifulSoup(response.content)
table = soup.find("table", class_="standard sortable").find_all("tr")[1:]
table = rebuild_table_as_list(table)
city_million = []
for city in table:
    city_million.append(city[1])
print(city_million)
print(len(city_million))
sql_string = ''''''
for element_num in range(len(city_million)):
    if element_num != (len(city_million) - 1):
        sql_string = sql_string + f'''a.name = '{city_million[element_num]}' OR ''' 
    else:
        sql_string = sql_string + f'''a.name = '{city_million[element_num]}' '''
print(sql_string)

['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж[a]', 'Пермь', 'Волгоград']
16
a.name = 'Москва' OR a.name = 'Санкт-Петербург' OR a.name = 'Новосибирск' OR a.name = 'Екатеринбург' OR a.name = 'Казань' OR a.name = 'Нижний Новгород' OR a.name = 'Челябинск' OR a.name = 'Красноярск' OR a.name = 'Самара' OR a.name = 'Уфа' OR a.name = 'Ростов-на-Дону' OR a.name = 'Омск' OR a.name = 'Краснодар' OR a.name = 'Воронеж[a]' OR a.name = 'Пермь' OR a.name = 'Волгоград' 


In [42]:
# текст запроса
query_5_7 = f'''with yandex_city_mln as (SELECT
                v.id as vid,
                v.area_id
            FROM
                employers e
                JOIN vacancies v  ON v.employer_id  = e.id 
            WHERE e.name = 'Яндекс' 
            GROUP BY vid, e.area
            )
            ,
            final as (SELECT 
                a.name,
                count(area_id) as cnt
            FROM 
                yandex_city_mln y
                JOIN areas a ON y.area_id = a.id 
            WHERE ({sql_string})
            GROUP BY a.name)

            SELECT
                *
            FROM 
                final
            
            UNION ALL
            SELECT
             'Total',
                SUM(cnt) as cnt
            FROM 
                final
         '''

In [43]:
# результат запроса
df_5_7 = pd.read_sql_query(query_5_7, connection)
print(df_5_7)

               name    cnt
0         Волгоград   24.0
1      Екатеринбург   39.0
2            Казань   25.0
3         Краснодар   30.0
4        Красноярск   23.0
5            Москва   54.0
6   Нижний Новгород   36.0
7       Новосибирск   35.0
8              Омск   21.0
9             Пермь   25.0
10   Ростов-на-Дону   25.0
11           Самара   26.0
12  Санкт-Петербург   42.0
13              Уфа   26.0
14        Челябинск   22.0
15            Total  453.0


C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_23508\300231338.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5_7 = pd.read_sql_query(query_5_7, connection)


***

<h3>Вывод по анализу работодателей</h3>

Из выполненных запросов мы можем сделать следующий вывод.
<p>
1. Компания Яндекс имеет самое большое количество вакансий среди работодателей, <br> так же как и количество регионов в которых эта компания разместила вакансии. 
даже не зная что такое яндекс можем прдеположить, что это крупная компания.<br> Подтверждением этого служет запрос по количество вакансий в разлиынх городах миллионниках.<br>
Они более мение равномерно одинаково распределны. <i>(исключение Москва, Санкт-Петербург и Екатеренбург)</i><br>
2. ~ 35% компании не уазали сферу дейтельности <br>
3. ~ 15% компаний занимаются разработкой программного обеспечения.<br>
</p>

<i>Если какие то запросы сдесь не учавстовали в выодах мне просто нечего о них сказать.</i>



In [44]:
# выводы по анализу работодателей


# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [45]:
# текст запроса
query_6_1 = f'''with data_like as (SELECT
                v.name,
                count(v.name) as cnt
            FROM
                vacancies v
            WHERE (name ilike '%data%' or name ilike '%данн%')
            GROUP BY v.name)

            SELECT
                SUM(cnt) as sum
            FROM 
                data_like
         '''

In [46]:
# результат запроса
df_6_1 = pd.read_sql_query(query_6_1, connection)
print(df_6_1)

C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_23508\2355451921.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_6_1 = pd.read_sql_query(query_6_1, connection)


      sum
0  1771.0


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [47]:
# текст запроса
main_DS_request = '''name ilike '%data scientist%' or name ilike '%data science%' or 
                    name ilike '%исследователь данных%' or name ilike '%machine learning%' or 
                    name ilike '%машинн%обучен%' or (name like '%ML%' and name not ilike '%HTML%' )'''

query_6_2 = f'''with data_like as (SELECT
                v.name,
                count(v.name) as cnt
            FROM
                vacancies v
            WHERE (({main_DS_request}) AND (name ilike '%junior%' OR experience ilike '%нет опыта%' OR employment  ilike '%стажировка%'))
            GROUP BY v.name)

            SELECT
                SUM(cnt) as sum
                
            FROM
                data_like
         '''


In [48]:
# результат запроса
df_6_2 = pd.read_sql_query(query_6_2, connection)
print(df_6_2)

C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_23508\2221805990.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_6_2 = pd.read_sql_query(query_6_2, connection)


    sum
0  51.0


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [49]:
# текст запроса
query_6_3 = f'''with data_like as (SELECT
                v.name as name, 
                count(v.name) as cnt,
                v.key_skills as skills     
            FROM
                vacancies v
            WHERE (({main_DS_request}))
            GROUP BY v.name, v.key_skills)

        
            SELECT
                SUM(cnt) as sum
            FROM
                data_like
            WHERE (skills ilike '%sql%' OR skills ilike '%postgres%')    
         '''

In [50]:
# результат запроса
df_6_3 = pd.read_sql_query(query_6_3, connection)
print(df_6_3)


C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_23508\3647444041.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_6_3 = pd.read_sql_query(query_6_3, connection)


     sum
0  201.0


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [51]:
# текст запроса
query_6_4 = f'''with data_like as (SELECT
                v.name as name, 
                count(v.name) as cnt,
                v.key_skills as skills     
            FROM
                vacancies v
            WHERE (({main_DS_request}))
            GROUP BY v.name, v.key_skills)

        
            SELECT
                SUM(cnt) as sum
            FROM
                data_like
            WHERE (skills ilike '%python%')    
         '''


In [52]:
# результат запроса
df_6_4 = pd.read_sql_query(query_6_4, connection)
print(df_6_4)

C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_23508\1617735037.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_6_4 = pd.read_sql_query(query_6_4, connection)


     sum
0  351.0


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [53]:
# текст запроса
query_6_5 = f'''with data_like as (SELECT
                v.name as name, 
                v.key_skills as skills     
            FROM
                vacancies v
            WHERE (({main_DS_request}))
            GROUP BY v.name, v.key_skills)

            SELECT
                name,
                skills,
                    length(skills) - length(replace(skills,CHR(9),'')) + 1 as cnt
            FROM
                data_like
            WHERE skills is not NULL
            GROUP BY name, skills
         '''



In [54]:
# результат запроса
df_6_5 = pd.read_sql_query(query_6_5, connection)
print("-- Вспомогательная инофрмация --")
print(df_6_5, '\n')

print("-- ответ --", '\n', round(df_6_5.mean(),2))


C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_23508\88080016.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_6_5 = pd.read_sql_query(query_6_5, connection)


-- Вспомогательная инофрмация --
                                                  name  \
0                                       Data Scientist   
1    Data Scientist/Machine Learning Engineer (Deve...   
2                                          ML Engineer   
3                               Data Scientist (Telco)   
4    Junior Data Scientist Управления модельных рис...   
..                                                 ...   
326                       Риск-менеджер/Data Scientist   
327                            Junior+ CV/ML developer   
328                               Data Scientist (NLP)   
329                                     Data Scientist   
330                 Инженер-программист/Data Scientist   

                                                skills  cnt  
0    Python\tSpark\tSQL\tDocker\tMachine Learning\t...    6  
1    Machine Learning\tDeep Learning\tData Science\...   10  
2                             Python\tGit\tLinux\tBash    4  
3    Python\tSQL\tData

C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_23508\88080016.py:6: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print("-- ответ --", '\n', round(df_6_5.mean(),2))


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [55]:
# текст запроса
query_6_6_part1 =  f'''with data_like as (SELECT
                v.name as name, 
                v.experience,
                v.salary_from,
                v.salary_to
            FROM
                vacancies v
            WHERE (({main_DS_request}))
            GROUP BY v.experience, v.name, v.salary_from, v.salary_to)
            '''
query_distinct_experience = query_6_6_part1 + f'''

            select distinct experience from data_like
            '''
query_6_6_part2 = f''',
            expirians_salary_notnull as (SELECT
                *
            FROM
                data_like
            WHERE (salary_from is NOT NULL OR salary_to IS NOT NULL)
            GROUP BY experience, name, salary_from, salary_to),

            expirians_solary_mean as
            (select
                experience,
                (coalesce(salary_to, 0) + coalesce(salary_from, 0)) / 2 as mean
            FROM
                expirians_salary_notnull
            WHERE (coalesce(salary_from, 0) != 0 AND coalesce(salary_to, 0) != 0)

            union all
            select
                experience,
                coalesce(salary_to, 0) 
            FROM
                expirians_salary_notnull
            WHERE coalesce(salary_from, 0) = 0

            UNION ALL 

            select
                experience,
                coalesce(salary_from, 0) 
            FROM
                expirians_salary_notnull
            WHERE coalesce(salary_to, 0) = 0)
            
            select 
                experience,
                ROUND(AVG(mean))
            from 
                expirians_solary_mean
            GROUP BY experience
'''

query_6_6  = query_6_6_part1 + query_6_6_part2 


In [56]:
# результат запроса
# Дополнительная таблица с опытом работы для наглядности 
df_distinct_experience = pd.read_sql_query(query_distinct_experience, connection)
display(df_distinct_experience)

# Основной ответ
df_6_6 = pd.read_sql_query(query_6_6, connection)
display(df_mean_DS_salary)

C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_23508\2798417237.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_distinct_experience = pd.read_sql_query(query_distinct_experience, connection)


,experience
0,От 3 до 6 лет
1,От 1 года до 3 лет
2,Более 6 лет
3,Нет опыта


C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_23508\2798417237.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_6_6 = pd.read_sql_query(query_6_6, connection)


NameError: name 'df_mean_DS_salary' is not defined

***

<h3> выводы по предметному анализу </h3>


<p> Выводы которые можно сделать, исходя из запросов по предметному анализу.<br>

1. Среди общего числа вакансий которые работают с данными составляют всего ~3.5% (1771 вакансий) от общего числа<br>
    А непосредственно DS меньше 1% (480). (К примеру "системных администраторов" в 1.5 раза больше)<br>
2. postgresql и sql у среди вакансий DS встречается в ~40% случаев, что дает понять что это не маловажный навык для DS.<br>
3. python у среди вакансий DS встречается в ~70% случаев, это значит один из основных навыков для данной професси.<br>
4. Общее число навыков указаных как основные у специалистов DS 6-7 (есть не точность ответ не совпал с курсом)<br>
5. В зависимости от опыты DS средняя зарплата специалиста ростет от ~40% до 50% (есть не точность ответ не совпал с курсом)
</p>


# Общий вывод по проекту

<h3> подведем итог исследования, обобщите выводы </h3>

<p>
По итогам можно сказать следующее:
Большенство вакансий сосредоточено в крупных городах, и средняя заработная плата там выше.
Так же большая вакансий расчитаны на полный рабочий день
На рынке труда очень много не опытных или мало опытных специалистов (для понимания причин у нас мало данных)

По работодателям.
В крупных компаниях по всем регионам, необходимы хорошие специалисты.
Ярким примером был "Яндекс" по которому мы проводили исследование.

Специалисты Data Scientist и Machine Learning являются достаточно востребованными. 
Многие навыки смежны с другими IT специалистами, но среднй уровень разоботных плат более привлекательный.
(сам являюсь системным администратором, имею представление об сфере IT)
</p>


In [57]:
# 
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

# дополнительные запросы которые помогли подвести выводы по предметному анализу
query_DS_total = f'''with data_like as (SELECT
                v.name as name, 
                count(v.name) as cnt,
                v.key_skills as skills     
            FROM
                vacancies v
            WHERE (({main_DS_request}))
            GROUP BY v.name, v.key_skills)

        
            SELECT
                SUM(cnt) as sum
            FROM
                data_like
         '''
df_DS_total = pd.read_sql_query(query_DS_total, connection)
display(df_DS_total)

main_SA_request = '''name ilike '%системный администратор%' or name ilike '%сисадмин%' or 
                    name ilike 'ИТ-инженер' or name ilike 'DevOps' '''

query_SA_total = f'''with data_like as (SELECT
                v.name as name, 
                count(v.name) as cnt,
                v.key_skills as skills     
            FROM
                vacancies v
            WHERE (({main_SA_request}))
            GROUP BY v.name, v.key_skills)

        
            SELECT
                name,
                SUM(cnt) as sum
            FROM
                data_like
                GROUP BY name
         '''
df_SA_total = pd.read_sql_query(query_SA_total, connection)
display(df_SA_total)

C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_23508\175538268.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_DS_total = pd.read_sql_query(query_DS_total, connection)


,sum
0,480.0


C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_23508\175538268.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_SA_total = pd.read_sql_query(query_SA_total, connection)


,name,sum
0,Системный администратор/специалист техподдержк...,1.0
1,Системный администратор-DevOps,1.0
2,"Системный администратор (Красноярск, ул. Спарт...",1.0
3,Devops/Системный администратор Linux,1.0
4,Системный администратор [FW8812],1.0
...,...,...
563,Системный администратор - сетевой инженер,1.0
564,Системный администратор СХД,1.0
565,Системный администратор\ Junior DevOps инженер,1.0
566,Системный администратор в типографию,1.0
